In [1]:
include("../src/Utero.jl")
using .Utero 

In [2]:
ctx = ⬅Context()
ForwardBackward!(ctx, x -> 2.0*x + x, AddParams!(ctx, 7.0))

(21.0, Any[3.0])

In [3]:
# should be 12.8
ctx = ⬅Context()
ForwardBackward!(ctx, x -> x^2.0 + x^2.0, AddParams!(ctx, 3.2)) 

(20.480000000000004, Any[12.8])

In [4]:
# Gradient should be 56.21
ctx = Utero.⬅Context()
ForwardBackward!(ctx, w -> 2.0 * (((2.0*w + 12.0) - w) * w) + sin(w^2.0), AddParams!(ctx, 7.0))

(265.0462473472405, Any[56.20829561241092])

In [5]:
# should be approx. (27.53, 3.0428)
ctx = Utero.⬅Context()
x, y = AddParams!(ctx, 13.2, 89.0)
ForwardBackward!(ctx, (x, y) -> (cos(y)^3 * 4sin(x))^2 + cos(x*y)^2, x, y) 

(1.0748674472530375, Any[27.53002464314667, 3.0428042273213602])

In [6]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(30,1))
(out, grads) = ForwardBackward!(ctx, _ -> W[5:5]' * W[5:5], 3)

(0.9685987651966126, Any[sparse([5], [1], [1.9683483077917003], 30, 1)])

In [7]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(30,1))
(out, grads) = ForwardBackward!(ctx, _ -> W' * W, 3)
print(size(grads[1]))

(30, 1)

In [8]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(30,1))
(out, grads) = ForwardBackward!(ctx, _ -> W' * W, 3)
print(size(grads[1]))

(30, 1)

In [9]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(2,1))
(out, grads) = ForwardBackward!(ctx, _ -> W .* 2, nothing)
@show size(W.val)
print(size(grads[1]))
print(grads[1])


size(W.val) = (2, 1)
(2, 1)

[2.0; 2.0;;]

In [10]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(2,1))
(out, grads) = ForwardBackward!(ctx, _ -> (W .* W)' * W, nothing)
@show size(W.val)
print(size(grads[1]))
print(grads[1])

usedsize(W.val) = 

(2, 1)
(2, 1)[1.4301627467083482; 0.256425914367678;;]

In [11]:
ctx = Utero.⬅Context()
W1, W2, W3 = AddParams!(ctx, rand(30,30), rand(40,30), rand(30,40))

function model(X)
    return  ReLU(W3 * ReLU(W2 * ReLU(W1 * X)))
end

X = rand(1500, 30)
Y = mapslices(row -> cos.(row), X, dims=2) 

println(ctx.Params[1].val)
for (x, y) in zip(eachrow(X), eachrow(Y))
    println(MeanSquaredError(model(x), y).val)
    (out, grads) = ForwardBackward!(ctx, x -> MeanSquaredError(model(x), y), x)
    GradientStep!(0.0001, ctx.Params, grads)
    exit("")
end
println(ctx.Params[1].val)


[0.7733688279507425 0.538075460372197 0.6363593628471069 0.15064916378832582 0.26174996024277053 0.11786421003490888 0.1861009931066644 0.19422724029606953 0.6853552084697592 0.7303073602741508 0.06346878026519454 0.3793690207778113 0.4784138493293234 0.5452646277385105 0.27495055885659403 0.20072827603361632 0.018168526810797503 0.5658945908695985 0.715482172683454 0.10261397723357624 0.7015204773071271 0.9037167913806788 0.9167501511202699 0.8128490956171811 0.7143790863530258 0.683631052591606 0.6110080754815543 0.8444012085438855 0.6679971853781532 0.45448397627714054; 0.6169299875451815 0.5632609506102921 0.7212462567411698 0.08015730290956746 0.8895581044613754 0.7488096746912165 0.4214409198558827 0.1942923081174034 0.09202985723169588 0.3475902977518257 0.532514913759177 0.24046327430393688 0.8130087911365784 0.8962561329860071 0.14686424585222535 0.9477809160429937 0.36438478781328076 0.15745153575474946 0.5858999354545977 0.19248954660177775 0.2307407149647216 0.1563250161869

 0.27992798182998413 0.6318574044798205 0.15545980293005135 0.7415310567508564 0.26484693559166617 0.23947112952495764 0.06847684749273475 0.3875275961903639; 0.3057979867863674 0.5855779338559142 0.906452216119031 0.8344652455621897 0.9621685962451224 0.4701800773535796 0.9712480419320925 0.589731105363756 0.852835293353429 0.7308784050944533 0.3132039302214975 0.9310431673425523 0.3858368508592387 0.09501746214359952 0.359452710778956 0.5583872975761238 0.6761801179161946 0.7253363287587625 0.20638997504555923 0.015366093804762171 0.4098588794062564 0.12614695037922663 0.4369883942399061 0.6976614921523188 0.5907335251774959 0.12078770748441636 0.7381186757206114 0.6461112016166038 0.48838371294255045 0.44887573000650616; 0.5613993429491984 0.756379055431591 0.12257738836965659 0.7125105744151597 0.47147411948251006 0.12702088359274688 0.18721419663569394 0.7951669707421953 0.30276822441863027 0.7024315314874193 0.2806781491182765 0.10464004884584788 0.49824565715473057 0.57213712750

1.6328040458583003e8


MethodError: MethodError: Cannot `convert` an object of type String to an object of type Int32

Closest candidates are:
  convert(::Type{T}, !Matched::Ptr) where T<:Integer
   @ Base pointer.jl:23
  convert(::Type{T}, !Matched::T) where T<:Number
   @ Base number.jl:6
  convert(::Type{T}, !Matched::Number) where T<:Number
   @ Base number.jl:7
  ...


In [12]:
using MLDatasets

TrainData = MNIST(:train)
ValidData = MNIST(:test)

ctx = ⬅Context()

densenet = Sequential(Dense(28*28, 16, ReLU), Dense(16, 16, ReLU), Dense(16, 10, ReLU), Softmax)
#display(densenet(rand(28*28, 1)))
#display(MeanSquaredError(densenet(rand(28*28, 1)), rand(10, 1)))
AddParams!(ctx, densenet)
println(typeof(densenet.Sequence[1].W))
for (x, y) in DataIterator(TrainData.features, TrainData.targets)
    x = reshape(x, 28*28, 1)
    y = OneHot(Int64(y+1), 10)
    y = reshape(y, 10, 1)
    display(densenet.Sequence[1].W)
    (out, paramgrads) = ForwardBackward!(ctx, x -> MeanSquaredError(densenet(x), y), x)
    GradientStep!(0.0001, ctx.Params, paramgrads)
    display(paramgrads[1])
    display(densenet.Sequence[1].W)
    exit("")
end

Main.Utero.⬅Tracker{Matrix{Float64}}([-0.7963648359461852 -0.041055304086892695 … -0.869816665174318 -0.438442985534509; 2.8138752564995433 0.2879623409516686 … 0.19600661952558904 -0.7524922124014215; … ; -0.9455176151800705 -0.4947949478527671 … -1.0613480957472587 -0.05570014372417808; -1.2031669278077466 0.05228416828237846 … -1.3976592802002246 -1.1114714310203004], UUID("2ca13580-c201-11ee-3b78-557ef0b2ee16"), Any[], Any[])

Main.Utero.⬅Tracker{Matrix{Float64}}


used

DimensionMismatch: DimensionMismatch: A has dimensions (10,1) but B has dimensions (10,1)